In [7]:
# import libraries
import boto3, re, sys, math, json, os, sagemaker, urllib.request
from sagemaker import get_execution_role
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import Image
from IPython.display import display
from time import gmtime, strftime
from sagemaker.predictor import csv_serializer



In [8]:
#create the estimator image
role = get_execution_role()
my_region = boto3.session.Session().region_name # set the region of the instance
xgboost_container = sagemaker.image_uris.retrieve("xgboost", my_region, "latest")

In [9]:
#create an S3 bucket
bucket_name = 'liabilitybucket123147576575' # <--- CHANGE THIS VARIABLE TO A UNIQUE NAME FOR YOUR BUCKET
s3 = boto3.resource('s3')
try:
    if  my_region == 'us-east-1':
      s3.create_bucket(Bucket=bucket_name)
    else: 
      s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={ 'LocationConstraint': my_region })
    print('S3 bucket created successfully')
except Exception as e:
    print('S3 error: ',e)

S3 error:  An error occurred (BucketAlreadyOwnedByYou) when calling the CreateBucket operation: Your previous request to create the named bucket succeeded and you already own it.


In [10]:
#load data
try:
  model_data = pd.read_csv('./liability.csv')
  print('Success: Data loaded into dataframe.')
except Exception as e:
    print('Data load error: ',e)

Success: Data loaded into dataframe.


In [11]:
model_data.head(10)

,Outcome,Product,Priority,Reason,Type,Origin
0,1,4,1,3,0,3
1,1,1,0,3,1,1
2,1,6,2,3,1,0
3,1,7,2,3,3,2
4,1,6,1,3,3,2
5,1,7,0,3,1,3
6,1,5,0,3,0,1
7,1,6,1,3,4,2
8,1,3,1,3,3,0
9,0,2,1,5,3,0


In [12]:
#train/test split
train_data, test_data = np.split(model_data.sample(frac=1, random_state=1729), [int(0.8 * len(model_data))])
print(train_data.shape, test_data.shape)

(39, 6) (10, 6)


In [13]:
#define input data in right format
prefix = "PACKT-DEMO"
model_data.to_csv('train.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
s3_input_train = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket_name, prefix), content_type='csv')

In [14]:
#set up estimator
sess = sagemaker.Session()
xgb = sagemaker.estimator.Estimator(xgboost_container,role, instance_count=1, instance_type='ml.m4.xlarge',output_path='s3://{}/{}/output'.format(bucket_name, prefix),sagemaker_session=sess)
xgb.set_hyperparameters(max_depth=5,eta=0.2,gamma=4,min_child_weight=6,subsample=0.8,silent=0,objective='binary:logistic',num_round=100)

In [15]:
#fit model
xgb.fit({'train': s3_input_train})


2021-07-22 17:49:54 Starting - Starting the training job...
2021-07-22 17:50:18 Starting - Launching requested ML instancesProfilerReport-1626976194: InProgress
......
2021-07-22 17:51:19 Starting - Preparing the instances for training.........
2021-07-22 17:52:39 Downloading - Downloading input data...
2021-07-22 17:53:23 Training - Training image download completed. Training in progress..Arguments: train
[2021-07-22:17:53:24:INFO] Running standalone xgboost training.
[2021-07-22:17:53:24:INFO] Path /opt/ml/input/data/validation does not exist!
[2021-07-22:17:53:24:INFO] File size need to be processed in the node: 0.0mb. Available memory size in the node: 8396.24mb
[2021-07-22:17:53:24:INFO] Determined delimiter of CSV input is ','
[17:53:24] S3DistributionType set as FullyReplicated
[17:53:24] 49x5 matrix with 245 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[17:53:24] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 prun

In [16]:
#deploy
xgb_predictor = xgb.deploy(initial_instance_count=1,instance_type='ml.m4.xlarge')


-----!

In [19]:
from sagemaker.serializers import CSVSerializer

test_data_array = test_data.drop('Outcome', axis=1).values #load the data into an array
xgb_predictor.serializer = CSVSerializer() # set the serializer type
predictions = xgb_predictor.predict(test_data_array).decode('utf-8') # predict!
predictions_array = np.fromstring(predictions[1:], sep=',') # and turn the prediction into an array
print(predictions_array.shape)

(10,)


In [20]:
cm = pd.crosstab(index=test_data['Outcome'], columns=np.round(predictions_array), rownames=['Observed'], colnames=['Predicted'])
tn = cm.iloc[0,0]; fn = cm.iloc[1,0]; tp = cm.iloc[1,1]; fp = cm.iloc[0,1]; p = (tp+tn)/(tp+tn+fp+fn)*100
print("\n{0:<20}{1:<4.1f}%\n".format("Overall Classification Rate: ", p))
print("{0:<15}{1:<15}{2:>8}".format("Predicted", "No Purchase", "Purchase"))
print("Observed")
print("{0:<15}{1:<2.0f}% ({2:<}){3:>6.0f}% ({4:<})".format("No Purchase", tn/(tn+fn)*100,tn, fp/(tp+fp)*100, fp))
print("{0:<16}{1:<1.0f}% ({2:<}){3:>7.0f}% ({4:<}) \n".format("Purchase", fn/(tn+fn)*100,fn, tp/(tp+fp)*100, tp))

IndexError: single positional indexer is out-of-bounds

In [ ]:
len(model_data.iloc[0,:-2])

In [ ]:
print([0 for i in range(0,59)])